In [1]:
!pip install openai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPAPI = userdata.get('SERPAPI')

In [3]:
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain_openai import OpenAI

In [4]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY,
             temperature=0)

# Load Tools

In [6]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=70204e24e6838ecfb490014713b79b8584bd0c11c859d4a964ed5a006422ae2f
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [7]:
tools = load_tools(["serpapi", "llm-math"],
                   llm=llm,
                   serpapi_api_key = SERPAPI)

In [9]:
# tools # this will expose api key

In [10]:
# tools[1]

In [12]:
tools[1].name , tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

# Initialize the agent

In [13]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [15]:
agent.run("What is the weather in Elkart, Indiana?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a search engine to find the current weather in Elkart, Indiana.
Action: Search
Action Input: "weather in Elkart, Indiana"
Observation: {'type': 'weather_result', 'temperature': '36', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '86%', 'wind': '9 mph', 'location': 'Elkhart, IN', 'date': 'Friday 10:00 AM', 'weather': 'Mostly cloudy'}
Thought: I now know the current weather in Elkart, Indiana.
Final Answer: The current weather in Elkart, Indiana is mostly cloudy with a temperature of 36 degrees Fahrenheit, 0% chance of precipitation, 86% humidity, and 9 mph winds.

> Finished chain.


'The current weather in Elkart, Indiana is mostly cloudy with a temperature of 36 degrees Fahrenheit, 0% chance of precipitation, 86% humidity, and 9 mph winds.'

In [16]:
agent.run("What time is it right now?")



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Search
Action Input: "current time"
Observation: 7:33 AM
Thought: I now know the final answer
Final Answer: 7:33 AM

> Finished chain.


'7:33 AM'